# $T_1$ under repeated measurement using QI 2.0

Let's first do the necessary imports and login in the Quantum Inspire website. 

In [ ]:
from utility_functions import *
from qiskit import QuantumCircuit
import matplotlib.pyplot as plt

!qi login "https://api.quantum-inspire.com"

In [2]:
experiment_backend = "Starmon-7"
provider = QIProvider()
backend = provider.get_backend(name=experiment_backend)

Using the function new_day() will create a folder in the path specified in the utility_functions.py where we will save all our results.

In [ ]:
data_directory = f"[...]"

new_directory_path = create_new_data_folder(datadir=data_directory)

We can now create the circuit using Qiskit.

In [ ]:
nr_qubits = 7 # Number of qubits of Starmon-7 processor
nr_points = 15 # Number of points for the T1 experiment
total_time = 150e-6 # Total time of the T1 experiment in [s]

qubit_index = 0 # We choose qubit Q0

cycle_time = 20e-9 # cycle time of the Central Controller (CC) instrument
dt = total_time / nr_points # time interval between each measurement point
measurement_times = np.linspace(start = 0.0, stop = total_time, num = nr_points)

qc = QuantumCircuit(7, nr_points) # NOTE: we should put backend.num_qubits here
qc.x(qubit_index) # qubit initialization to the |1> state
qc.measure(qubit = qubit_index, cbit = 0) # initial measurement at time t=0
for time_idx in range(1, nr_points):
    qc.delay(duration = int((dt / cycle_time)), qarg = qubit_index) # delay in units of CC cycles
    qc.measure(qubit = qubit_index, cbit = time_idx)

If you want to see the circuit run the cell below.

In [ ]:
qc.draw('mpl')

Choose the name of the files that we will create and save. 

In [13]:
basename = 'Example'
suffix = 'T1_under_measurement_w5us'
histname, circuit_name, rawname = prepare_file(basename, suffix, doraw=1)

Specify the number of shots and run the algorithm.

In [14]:
nr_shots = 2**14 # Careful not to exceed the max_shots of the backend!

results = api_run_and_save(0, Qcircuit = qc, rawdata_filename = rawname, circuit_name=circuit_name,
                           histname = histname, shots=nr_shots, backend_name=experiment_backend, get_results = True,
                           get_hist_data=True, measurement_list=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], get_raw_data=True)

### Data analysis

The data are saved in a csv file in the format:

In [15]:
df = pd.read_csv(rawname+"_0.csv")
measurement_outcomes_list = np.zeros((len(df["Raw data values"]), (len(df["Raw data values"][0])-1)), dtype = np.int8)

for shot_index in range(len(df["Raw data values"])):
    for measurement_index in range(1, len(df["Raw data values"][0])):
        measurement_outcomes_list[shot_index][(measurement_index - 1)] = np.int8(df["Raw data values"][shot_index][len(df["Raw data values"][0]) - measurement_index])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3), sharey = 'row', sharex = 'col')
ax.plot(np.arange(len(np.sum(measurement_outcomes_list, 0))) * (dt * 1e6 + 1), np.sum(measurement_outcomes_list, 0) / nr_shots, '-o')

ax.set_ylim(0, 1)
ax.set_xlim(-1.5, max(np.arange(len(np.sum(measurement_outcomes_list, 0))) * (dt * 1e6 + 1)) + 1.5)

ax.set_title('$T_1$ under repeated measurement')
ax.set_ylabel('Fraction')
ax.set_xlabel('Time (us)')